In [1]:
from collections import deque

In [2]:
class ReserveAccount:
    """
    The reserves_account class generates a reserves_account object for a specified currency pair
    Reserves accounts are initialised with an order book that starts at ICO price unless otherwise specified, and a reserve balance of 0

    Reserve accounts have 6 function calls:
    - quote: returns the current marginal price and the order book volume at that price
    - update_supply: changes the supply factor variable that determines the order book volume available at each price band
    - issue: adds reserves to the reserve balance and subtracts corresponding orderbook volume
    - retire: subtracts reserves from the reserve balance and adds corresponding orderbook volume
    - refresh: refreshes the orderbook starting with the current price
    - print_full_book: prints the entire order book and reserve LIFO book
    """

    def __init__(self, currency_pair, supply_factor=1.0, start_price=0.0001, reserve_price=0.00005, reserve_balance=0.0, orderbook_len=5):
        """
        :currency_pair  : the currency pair that Holo Fuel is traded against at this account
        :supply_factor  : Each price tranch of the order book has a volume of supply_factor * 1,000,000 Fuel
        :start_price    : the starting lowest price offered by the reserve order book denominated in {currency_pair}
        :reserve_balance: sets the amount of {currency_pair} held in the reserve at start
        :orderbook_len  : the number of tranches to maintain in the orderbook
        """

        self.supply_factor = supply_factor # For P-D control over orderbook volumes
        self.current_price = start_price
        self.currency_pair = currency_pair
        self.orderbook_len = orderbook_len

        self.order_book_vol = deque(maxlen=self.orderbook_len)
        self.order_book_price = deque(maxlen=self.orderbook_len)
        self.reserves = deque() # Create reserve as double-ended queue and append tranches

        self.refresh()
        
        retire_price = reserve_price
        reserve_tranch = [retire_price, self.supply_factor * 1000000] # we keep vol and price together for reserves because unlike order book we can't shift one vs. the other
        self.reserves.appendleft(reserve_tranch)
        # The oldest price in the reserves will be on the right, and the newest will be on the left


    def update_supply(self, new_supply_factor=None):
        """
        This function is designed to rebuild the orderbook when changing the supply factor
        It functions mostly like the original order book builder - starting with current price and adding higher price tranches at % intervals
        A key difference is treatment of partial tranche volumes (i.e. where some volume has already been bought)
        We will move the partial tranche volume to the current price, and scale it by the change in the supply factor
        """
        old_supply_factor = self.supply_factor
        if new_supply_factor is not None:
            self.supply_factor = new_supply_factor

        current_tranch_vol = self.order_book_vol[-1]

        for ii in range(self.orderbook_len):
            issue_price = self.current_price * (1 + ii/100) # Reserve tranches need increment in % of price, otherwise at higher prices you get ludicrous volumes available before meaningful price change
            self.order_book_price.appendleft(issue_price)
            if ii == 0:
                self.order_book_vol.appendleft(self.supply_factor/old_supply_factor * current_tranch_vol)
            else:
                self.order_book_vol.appendleft(self.supply_factor * 1000000)


    def quote(self, order_type):
        """
        :order_type: 'buy' to buy fuel from reserve (issue) or 'sell' to sell and retire fuel at reserve
        """
        if order_type == 'buy':
            volume = self.order_book_vol[-1]
            price = self.order_book_price[-1]

        elif order_type == 'sell':
            volume = self.reserves[0][1]
            price = self.reserves[0][0]
        else:
            print('No order type specified. Please specify "buy" or "sell"')
            return

        return price, volume


    def issue(self, volume):
        """
        :volume: Volume of Fuel to be issued
        """
        sum_orderbook_vol = sum(self.order_book_vol)

        if volume > sum_orderbook_vol:
            print("Volume exceeds total orderbook volume")
            return

        # Update orderbook for purchases
        while volume > 0:
            quote_price, quote_vol = self.quote('buy')
            self.current_price = quote_price
            buy_vol = min(quote_vol, volume)
            remainder = quote_vol - buy_vol

            if remainder == 0:
                self.order_book_vol.pop()
                self.order_book_price.pop()
            else:
                self.order_book_vol[-1] = remainder
            
            if self.reserves[0][0] == self.current_price:
                self.reserves[0][1] += buy_vol
            else:
                self.reserves.appendleft([self.current_price, buy_vol])

            volume -= buy_vol

        # add new tranches to replace bought ones
        for ii in range(len(self.order_book_vol), self.orderbook_len):
            self.order_book_price.appendleft(self.current_price * (1 + ii/100))
            self.order_book_vol.appendleft(self.supply_factor * 1000000)


    def retire(self, volume):
        """
        :volume: Volume of Fuel to be retired
        """
        sum_reserve_vol = sum([tranche[1] for tranche in self.reserves])

        if volume > sum_reserve_vol:
            print("Volume exceeds total reserve volume")
            return

        # Update LIFO accounts
        while volume > 0:
            quote_price, quote_vol = self.quote('sell')
            self.current_price = quote_price
            sell_vol = min(quote_vol, volume)
            remainder = quote_vol - sell_vol

            if remainder == 0:
                self.reserves.popleft()
            else:
                self.reserves[0] = [self.current_price, remainder]

            # Add the retired volume to the order book
            if self.order_book_price[-1] == self.current_price:
                self.order_book_vol[-1] += sell_vol
            else:
                self.order_book_price.append(self.current_price)
                self.order_book_vol.append(sell_vol)
                
            volume -= sell_vol
    
    def refresh(self):
        self.order_book_price.clear()
        self.order_book_vol.clear()
        
        for ii in range(self.orderbook_len):
            issue_price = self.current_price * (1 + ii/100) # Reserve tranches need increment in % of price, otherwise at higher prices you get ludicrous volumes available before meaningful price change
            self.order_book_price.appendleft(issue_price)
            self.order_book_vol.appendleft(self.supply_factor * 1000000)
            # The cheapest price in the order book will be on the right, and the most expensive will be on the left

    def print_full_book(self):
        for ii in range(len(self.order_book_vol)):
            print("Issue: {} Fuel @ Price of {:.5f} {}".format(self.order_book_vol[ii], self.order_book_price[ii], self.currency_pair))

        print("===============================================")

        for ii in range(len(self.reserves)):
            print("Buy-Back: {} Fuel @ Price of {:.5f} {}".format(self.reserves[ii][1], self.reserves[ii][0], self.currency_pair))

In [3]:
ETH_USD_price = 400
FUEL_USD_price = 0.001
FUEL_ETH_price = FUEL_USD_price / ETH_USD_price


# Create 2 reserves
USD_reserve = ReserveAccount(currency_pair='USD',start_price=FUEL_USD_price, reserve_price=FUEL_USD_price*0.99)
ETH_reserve = ReserveAccount(currency_pair='ETH',start_price=FUEL_ETH_price, reserve_price=FUEL_ETH_price*0.99)

### Check reserve class works

In [4]:
# Check 1: Does the reserve have an order book and LIFO 
USD_reserve.print_full_book()

Issue: 1000000.0 Fuel @ Price of 0.00104 USD
Issue: 1000000.0 Fuel @ Price of 0.00103 USD
Issue: 1000000.0 Fuel @ Price of 0.00102 USD
Issue: 1000000.0 Fuel @ Price of 0.00101 USD
Issue: 1000000.0 Fuel @ Price of 0.00100 USD
Buy-Back: 1000000.0 Fuel @ Price of 0.00099 USD


In [5]:
# Check 2: If we buy and sell the same amount but in different individual orders we return to the original order book
USD_reserve.issue(5000)
USD_reserve.issue(2100000)
USD_reserve.retire(1100000)
USD_reserve.retire(1000000)
USD_reserve.retire(5000)

In [6]:
USD_reserve.print_full_book()

Issue: 1000000.0 Fuel @ Price of 0.00104 USD
Issue: 1000000.0 Fuel @ Price of 0.00103 USD
Issue: 1000000.0 Fuel @ Price of 0.00102 USD
Issue: 1000000.0 Fuel @ Price of 0.00101 USD
Issue: 1000000.0 Fuel @ Price of 0.00100 USD
Buy-Back: 1000000.0 Fuel @ Price of 0.00099 USD


In [7]:
# Check 3: Does doubling the supply factor double the order book volume while keeping it constant
USD_reserve.update_supply(new_supply_factor=2)
USD_reserve.print_full_book()

Issue: 2000000 Fuel @ Price of 0.00104 USD
Issue: 2000000 Fuel @ Price of 0.00103 USD
Issue: 2000000 Fuel @ Price of 0.00102 USD
Issue: 2000000 Fuel @ Price of 0.00101 USD
Issue: 2000000.0 Fuel @ Price of 0.00100 USD
Buy-Back: 1000000.0 Fuel @ Price of 0.00099 USD


In [8]:
# Check 4: If we keep the supply factor static and repeatedly update supply it should do nothing
USD_reserve.update_supply(new_supply_factor=2)
USD_reserve.update_supply(new_supply_factor=2)
USD_reserve.update_supply(new_supply_factor=2)
USD_reserve.update_supply(new_supply_factor=2)
USD_reserve.update_supply(new_supply_factor=2)
USD_reserve.update_supply(new_supply_factor=2)
USD_reserve.update_supply(new_supply_factor=2)
USD_reserve.update_supply(new_supply_factor=2)
USD_reserve.print_full_book()

Issue: 2000000 Fuel @ Price of 0.00104 USD
Issue: 2000000 Fuel @ Price of 0.00103 USD
Issue: 2000000 Fuel @ Price of 0.00102 USD
Issue: 2000000 Fuel @ Price of 0.00101 USD
Issue: 2000000.0 Fuel @ Price of 0.00100 USD
Buy-Back: 1000000.0 Fuel @ Price of 0.00099 USD
